In [2]:
import pandas as pd 
from glob import glob
import os
import office365
import requests
import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings("ignore")

In [3]:
# path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
# f = glob("Dthu\*csv")
# files_Dthu_parquet = glob(os.path.join(path, "dthu_parquet_files\\*.parquet"))
# files_CP = glob(os.path.join(path, "CP\\*.csv"))
# files_Others = glob(os.path.join(path, "Others\\*.csv"))
# files_SLBan = glob(os.path.join(path, "SLBan\\*.csv"))
# files_TK = glob(os.path.join(path, "TK\\*.csv"))
dthu = r'D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\dthu.parquet'

# Ke Hoach

In [142]:
import pandas as pd
from glob import glob
import os
import requests

path = r"D:\pnj.com.vn\HuynhTN - Documents\2025\Planning"
kehoach = os.path.join(path, "Planning2025.xlsx")
kehoach = pd.read_excel(kehoach, sheet_name="Budget_Total", header=1)
kehoach = kehoach[kehoach["Tháng"].isin(range(1, 13))]
col = ["T", "9999"]
kehoach = kehoach.drop(columns=col)
# unstack the dataframe keep 3 first columns name level_3 is Plant Code
kehoach = kehoach.set_index(['Chi Phí', 'Chi tiết', 'Tháng']).stack().reset_index(name='Amt')
kehoach = kehoach.rename(columns={"level_3": "Plant Code"})
kehoach.groupby(["Chi Phí", "Chi tiết"])["Amt"].sum().reset_index()
kehoach["Tháng"] = pd.to_datetime(kehoach["Tháng"].astype('str') + "-2025").dt.to_period('M')

def get_kehoach(kehoach, chi_phi, chi_tiet, thang, nam):
    return kehoach[(kehoach["Chi Phí"] == chi_phi) & (kehoach["Chi tiết"] == chi_tiet) & (kehoach["Tháng"].dt.month.isin([thang])) & (kehoach["Tháng"].dt.year.isin([nam]))]["Amt"].sum()

dthu_tsv = get_kehoach(kehoach, "Doanh Thu", "TSV", 1, 2025)
dthu_vm = get_kehoach(kehoach, "Doanh Thu", "VM", 1, 2025)
lgop_tsv = get_kehoach(kehoach, "Lãi Gộp", "TSV", 1, 2025)
lgop_vm = get_kehoach(kehoach, "Lãi Gộp", "VM", 1, 2025)
cpv_tsv = get_kehoach(kehoach, "Chi Phí Vốn", "TSV", 1, 2025)
cpv_vm = get_kehoach(kehoach, "Chi Phí Vốn", "VM", 1, 2025)
cp_luong = get_kehoach(kehoach, "Chi Phí Lương", "Chi Phí Lương", 1, 2025)
cp_mb =  get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP Mặt Bằng", 1, 2025)
cp_tscd = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP Phân Bổ (KH)", 1, 2025)
cp_ccdc = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP Phân Bổ (CCDC)", 1, 2025)
cp_sc = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP Phân Bổ (SC)", 1, 2025)
cp_ct = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP chuyển tiền", 1, 2025)
cp_cl = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP còn lại", 1, 2025)
cp_hc = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP hành chính", 1, 2025)
cp_km = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP khuyến mãi", 1, 2025)
cp_mkt = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP marketing", 1, 2025)
cp_vlp = get_kehoach(kehoach, "Chi Phí Ngoài Lương", "CP vật liệu phụ", 1, 2025)

print(dthu_tsv, dthu_vm, lgop_tsv, lgop_vm, cpv_tsv, cpv_vm, cp_luong, cp_mb, cp_tscd, cp_ccdc, cp_sc, cp_ct, cp_cl, cp_hc, cp_km, cp_mkt, cp_vlp)

213697.16936417203 62120.557371767216 55443.44828722369 1975.2588326694186 0.0 0.0 12218.899522866906 2857.8796092055554 142.95140845238095 749.0885938472221 1224.670401 512.8732064740129 1148.1176679480575 865.5863376262499 0.0 282.66666666666663 492.4708638632193


# Tonkho

In [ ]:
import pandas as pd 
from glob import glob
import os
import requests
import streamlit as st
import helper

import warnings
warnings.filterwarnings("ignore")

parquet = r"https://raw.githubusercontent.com/Huynh-Tr/report/main/tonkho.parquet"

def tkho():
    df = pd.read_parquet(parquet)
    df = df[df["Month year"] != "Month year"]
    df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
    df["Plant Code"] = df["Plant Code"].astype('str')
    lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
    lv3 = ["CAO"]
    lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
    cols = ["Month year", "Plant Code", "Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", \
        "MM. Inventory Qty", "MM. Inventory Amt (Adjust)", "MM.Avg Inventory Qty", "MM.Avg Inventory Amt (Adjust)", \
        "FM 10_Inventory Cost", "MM. Repurchasing Amt", "MM. Repurchasing Qty"]

    df_tk_tsv = df.copy()
    df_tk_tsv = df_tk_tsv[~df_tk_tsv["Product Group 2 Name"].isin(lv2)]
    df_tk_tsv = df_tk_tsv[~df_tk_tsv["Product Group 3 Name"].isin(lv3)]
    df_tk_tsv = df_tk_tsv[~df_tk_tsv["Product Group 4 Name"].isin(lv4)]
    df_tk_tsv = df_tk_tsv[cols]

    df_tk_vm = df.copy()
    df_tk_vm = df_tk_vm[df_tk_vm["Product Group 4 Name"].isin(lv4)]
    df_tk_vm = df_tk_vm[cols]

    return df_tk_tsv, df_tk_vm

def chitieu_theoky(df, col, year, month, plant_code):
    if plant_code == 'Select All':
        chitieu_th = df[(df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck = df[(df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9
        chitieu_lk = df[(df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck_lk = df[(df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9

    else:
        chitieu_th = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9
        chitieu_lk = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck_lk = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9

    return chitieu_th, chitieu_ck, chitieu_lk, chitieu_ck_lk

def chitieu(year, month, plant_code='Select All'):
    df_tk_tsv, df_tk_vm = tkho()
    if plant_code == 'Select All':
        # chi phi von
        sum_cpv_tsv, sum_cpv_tsv_ck, sum_cpv_tsv_lk, sum_cpv_tsv_ck_lk = chitieu_theoky(df=df_tk_tsv, col="FM 10_Inventory Cost", year=year, month=month, plant_code=plant_code)
        sum_cpv_vm, sum_cpv_vm_ck, sum_cpv_vm_lk, sum_cpv_vm_ck_lk = chitieu_theoky(df=df_tk_vm, col="FM 10_Inventory Cost", year=year, month=month, plant_code=plant_code)
        # ton von
        sum_tv_tsv, sum_tv_tsv_ck, sum_tv_tsv_lk, sum_tv_tsv_ck_lk = chitieu_theoky(df=df_tk_tsv, col="MM.Avg Inventory Amt (Adjust)", year=year, month=month, plant_code=plant_code)
        sum_tv_vm, sum_tv_vm_ck, sum_tv_vm_lk, sum_tv_vm_ck_lk = chitieu_theoky(df=df_tk_vm, col="MM.Avg Inventory Amt (Adjust)", year=year, month=month, plant_code=plant_code)

    else:
        # chi phi von
        sum_cpv_tsv, sum_cpv_tsv_ck, sum_cpv_tsv_lk, sum_cpv_tsv_ck_lk = chitieu_theoky(df=df_tk_tsv, col="FM 10_Inventory Cost", year=year, month=month, plant_code=plant_code)
        sum_cpv_vm, sum_cpv_vm_ck, sum_cpv_vm_lk, sum_cpv_vm_ck_lk = chitieu_theoky(df=df_tk_vm, col="FM 10_Inventory Cost", year=year, month=month, plant_code=plant_code)
        # ton von
        sum_tv_tsv, sum_tv_tsv_ck, sum_tv_tsv_lk, sum_tv_tsv_ck_lk = chitieu_theoky(df=df_tk_tsv, col="MM.Avg Inventory Amt (Adjust)", year=year, month=month, plant_code=plant_code)
        sum_tv_vm, sum_tv_vm_ck, sum_tv_vm_lk, sum_tv_vm_ck_lk = chitieu_theoky(df=df_tk_vm, col="MM.Avg Inventory Amt (Adjust)", year=year, month=month, plant_code=plant_code)

    result_tkho = pd.DataFrame(
        {
        'Chỉ Tiêu': ['Chi Phí Vốn', 'TSV', 'VM'],
        '': ['', '', ''],
        '  Thực Hiện  ': [sum_cpv_tsv + sum_cpv_vm, sum_cpv_tsv, sum_cpv_vm],
        '  Cùng Kỳ    ': [sum_cpv_tsv_ck + sum_cpv_vm_ck, sum_cpv_tsv_ck, sum_cpv_vm_ck],
        '  Kế Hoạch   ': [sum_cpv_tsv + sum_cpv_vm, sum_cpv_tsv, sum_cpv_vm],
        ' ': ['', '', ''],
        ' LK Thực Hiện': [sum_cpv_tsv_lk + sum_cpv_vm_lk, sum_cpv_tsv_lk, sum_cpv_vm_lk],
        ' LK Cùng Kỳ  ': [sum_cpv_tsv_ck_lk + sum_cpv_vm_ck_lk, sum_cpv_tsv_ck_lk, sum_cpv_vm_ck_lk],
        ' LK Kế Hoạch ': [sum_cpv_tsv + sum_cpv_vm, sum_cpv_tsv, sum_cpv_vm],
        '  ': ['', '', ''],
        'LK Thực Hiện ': [sum_cpv_tsv_lk + sum_cpv_vm_lk, sum_cpv_tsv_lk, sum_cpv_vm_lk],
        '       KH Năm': [sum_cpv_tsv + sum_cpv_vm, sum_cpv_tsv, sum_cpv_vm]
        }
    )

    return result_tkho

result_tkho = chitieu(year=[2021], month=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], plant_code='Select All')

SyntaxError: invalid syntax (4256108283.py, line 37)

# Cphi

In [ ]:
df_cp = pd.read_parquet(r"https://raw.githubusercontent.com/Huynh-Tr/report/main/cphi.parquet")
# df_cp = df[df["Month year"] != "Month year"]
df_cp["Month year"] = pd.to_datetime(df_cp["Month year"]).dt.to_period('M')

# create MaCH column with condition right 4 chartacters of cost center code if not null else right 4 characters of GL Account Code
df_cp["Plant Code"] = df_cp["Cost Center Code"].fillna(df_cp["Plant Code"].astype('str')).astype('int').astype('str').str[-4:] \
    .replace('0000', 'P015').replace('9999', 'P015')

df_cp["FM. Loc Amt"] = df_cp["FM. Loc Amt"].astype('float')

cols = ["Month year", "GL Account Code", "GL LV1 Name", "FM. Loc Amt", "PnL Name", "MaCH"]
gl_code_exl = 64110120 # list of GL Account Code
gl_code_range = [64100000, 64252100]

# filter data cols with gl_code_exl and in range gl_code_range
df_cp = df_cp[cols][(df_cp["GL Account Code"] != gl_code_exl) & (df_cp["GL Account Code"].between(gl_code_range[0], gl_code_range[1]))]

df_cp.head()

,Month year,GL Account Code,GL LV1 Name,FM. Loc Amt,PnL Name,MaCH
0,2022-01,64130110,CP CCDC,-552500.0,Chi phí bán hàng,1013
1,2022-01,64130140,CP CCDC,-4387500.0,Chi phí bán hàng,1013
2,2022-01,64140110,CP KH TSCĐ,-3336389.0,Chi phí bán hàng,1013
3,2022-01,64140130,CP KH TSCĐ,-1533889.0,Chi phí bán hàng,1013
4,2022-01,64110100,CP Nhân viên,-111131428.0,Chi phí bán hàng,1115


In [3]:
list(range(4, 13))

[4, 5, 6, 7, 8, 9, 10, 11, 12]

In [57]:
df_cp["GL LV1 Name"].unique()

array(['CP CCDC', 'CP KH TSCĐ', 'CP Nhân viên', 'CP vật liệu phụ',
       'CP Còn lại', 'CP Thuê ngoài', 'CP hành chính', 'CP chuyển tiền',
       'CP Marketing & Khuyến mãi', 'CP sửa chữa, bảo trì'], dtype=object)

In [74]:
df_cp = pd.concat([pd.read_csv(file, encoding='utf-8', sep=',', header=0) for file in files_CP])
df_cp["Month year"] = pd.to_datetime(df_cp["Month year"]).dt.to_period('M')

# create MaCH column with condition right 4 chartacters of cost center code if not null else right 4 characters of GL Account Code
df_cp["MaCH"] = df_cp["Cost Center Code"].fillna(df_cp["Plant Code"].astype('str')).astype('int').astype('str').str[-4:] \
    .replace('0000', 'P015').replace('9999', 'P015')


cols = ["Month year", "GL Account Code", "GL LV1 Name", "FM. Loc Amt", "PnL Name", "MaCH"]
gl_code_exl = 64110120 # list of GL Account Code
gl_code_range = [64100000, 64252100]

# filter data cols with gl_code_exl and in range gl_code_range
df_cp = df_cp[cols][(df_cp["GL Account Code"] != gl_code_exl) & (df_cp["GL Account Code"].between(gl_code_range[0], gl_code_range[1]))]

df_cp.head()

,Month year,GL Account Code,GL LV1 Name,FM. Loc Amt,PnL Name,MaCH
0,2022-01,64130110,CP CCDC,-552500.0,Chi phí bán hàng,1013
1,2022-01,64130140,CP CCDC,-4387500.0,Chi phí bán hàng,1013
2,2022-01,64140110,CP KH TSCĐ,-3336389.0,Chi phí bán hàng,1013
3,2022-01,64140130,CP KH TSCĐ,-1533889.0,Chi phí bán hàng,1013
4,2022-01,64110100,CP Nhân viên,-111131428.0,Chi phí bán hàng,1115


In [ ]:
# group by GL LV1 Name and Month year
df_cp[df_cp["Month year"]=="2025-01"].groupby(["GL LV1 Name", "Month year"])["FM. Loc Amt"].sum() / 1e9

GL LV1 Name                Month year
CP CCDC                    2025-01      -0.744826
CP Còn lại                 2025-01      -0.817176
CP KH TSCĐ                 2025-01      -0.211752
CP Marketing & Khuyến mãi  2025-01      -0.007248
CP Nhân viên               2025-01      -8.924364
CP Thuê ngoài              2025-01      -2.867169
CP chuyển tiền             2025-01      -0.529042
CP hành chính              2025-01      -0.442921
CP sửa chữa, bảo trì       2025-01      -1.238189
CP vật liệu phụ            2025-01      -0.493743
Name: FM. Loc Amt, dtype: float64

# Dthu

In [5]:
import pandas as pd 
from glob import glob
import os
import requests
import streamlit as st
import helper

import warnings
warnings.filterwarnings("ignore")

# parquet = r"https://raw.githubusercontent.com/Huynh-Tr/report/main/dthu.parquet"
parquet = r"D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\dthu.parquet"

def dthu():
    df = pd.read_parquet(parquet)
    df = df[df["Month year"] != "Month year"]
    df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
    df["Plant Code"] = df["Plant Code"].astype('str')
    lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
    lv3 = ["CAO"]
    lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG KHÔNG ÉP SIÊU", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
    account_code = [51131000, 51132000]
    col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", \
        "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]

    df_tsv = df.copy()
    df_tsv = df_tsv[~df_tsv["Product Group 2 Name"].isin(lv2)]
    df_tsv = df_tsv[~df_tsv["Product Group 3 Name"].isin(lv3)]
    df_tsv = df_tsv[~df_tsv["Product Group 4 Name"].isin(lv4)]
    df_tsv = df_tsv[~df_tsv["GL Account Code"].isin(account_code)]
    df_tsv = df_tsv[col]

    df_vm = df.copy()
    df_vm = df_vm[df_vm["Product Group 4 Name"].isin(lv4)]
    df_vm = df_vm[col]

    return df_tsv, df_vm

def chitieu_theoky(df, col, year, month, plant_code):
    if plant_code == 'Select All':
        chitieu_th = df[(df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck = df[(df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9
        chitieu_lk = df[(df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck_lk = df[(df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9

    else:
        chitieu_th = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(month)) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9
        chitieu_lk = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin(year))][col].sum() / 1e9
        chitieu_ck_lk = df[(df["Plant Code"].isin(plant_code)) & (df["Month year"].dt.month.isin(range(1, max(month) + 1))) & (df["Month year"].dt.year.isin([max(year) - 1]))][col].sum() / 1e9

    return chitieu_th, chitieu_ck, chitieu_lk, chitieu_ck_lk

def chitieu(year, month, plant_code='Select All'):    
    data_tsv, data_vm = dthu()
    if plant_code == 'Select All':
        sum_fm_tsv01, sum_fm_tsv01_ck, sum_fm_tsv01_lk, sum_fm_tsv01_ck_lk = chitieu_theoky(df=data_tsv, col="FM 01_Invoices Revenue", year=year, month=month, plant_code=plant_code)
        sum_fm_vm01, sum_fm_vm01_ck, sum_fm_vm01_lk, sum_fm_vm01_ck_lk = chitieu_theoky(df=data_vm, col="FM 01_Invoices Revenue", year=year, month=month, plant_code=plant_code)                  
        sum_fm_tsv07, sum_fm_tsv07_ck, sum_fm_tsv07_lk, sum_fm_tsv07_ck_lk = chitieu_theoky(df=data_tsv, col="FM 07_Gross Profit", year=year, month=month, plant_code=plant_code)
        sum_fm_vm07, sum_fm_vm07_ck, sum_fm_vm07_lk, sum_fm_vm07_ck_lk = chitieu_theoky(df=data_vm, col="FM 07_Gross Profit", year=year, month=month, plant_code=plant_code)

    else:
        sum_fm_tsv01, sum_fm_tsv01_ck, sum_fm_tsv01_lk, sum_fm_tsv01_ck_lk = chitieu_theoky(df=data_tsv, col="FM 01_Invoices Revenue", year=year, month=month, plant_code=plant_code)
        sum_fm_vm01, sum_fm_vm01_ck, sum_fm_vm01_lk, sum_fm_vm01_ck_lk = chitieu_theoky(df=data_vm, col="FM 01_Invoices Revenue", year=year, month=month, plant_code=plant_code)                  
        sum_fm_tsv07, sum_fm_tsv07_ck, sum_fm_tsv07_lk, sum_fm_tsv07_ck_lk = chitieu_theoky(df=data_tsv, col="FM 07_Gross Profit", year=year, month=month, plant_code=plant_code)
        sum_fm_vm07, sum_fm_vm07_ck, sum_fm_vm07_lk, sum_fm_vm07_ck_lk = chitieu_theoky(df=data_vm, col="FM 07_Gross Profit", year=year, month=month, plant_code=plant_code)

    print('sum_fm_tsv01:', sum_fm_tsv01)
    result_dthu = pd.DataFrame(
        {
        'Chỉ Tiêu': ['Doanh thu', 'TSV', 'VM', 'Lãi Gộp', 'TSV', 'VM'],
        '': ['', '', '', '', '', ''],
        '  Thực Hiện  ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
        '  Cùng Kỳ    ': [(sum_fm_tsv01_ck + sum_fm_vm01_ck), (sum_fm_tsv01_ck), (sum_fm_vm01_ck), (sum_fm_tsv07_ck + sum_fm_vm07_ck), (sum_fm_tsv07_ck), (sum_fm_vm07_ck)],
        '  Kế Hoạch   ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
        ' ': ['', '', '', '', '', ''],
        ' LK Thực Hiện': [(sum_fm_tsv01_lk + sum_fm_vm01_lk), (sum_fm_tsv01_lk), (sum_fm_vm01_lk), (sum_fm_tsv07_lk + sum_fm_vm07_lk), (sum_fm_tsv07_lk), (sum_fm_vm07_lk)],
        ' LK Cùng Kỳ  ': [(sum_fm_tsv01_ck_lk + sum_fm_vm01_ck_lk), (sum_fm_tsv01_ck_lk), (sum_fm_vm01_ck_lk), (sum_fm_tsv07_ck_lk + sum_fm_vm07_ck_lk), (sum_fm_tsv07_ck_lk), (sum_fm_vm07_ck_lk)],
        ' LK Kế Hoạch ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
        '  ': ['', '', '', '', '', ''],
        'LK Thực Hiện ': [(sum_fm_tsv01_lk + sum_fm_vm01_lk), (sum_fm_tsv01_lk), (sum_fm_vm01_lk), (sum_fm_tsv07_lk + sum_fm_vm07_lk), (sum_fm_tsv07_lk), (sum_fm_vm07_lk)],
        '       KH Năm': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)]
        }
    )    

    return result_dthu

result_dthu = chitieu(year=[2024], month=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], plant_code='Select All')

sum_fm_tsv01: 1999.337113249


In [ ]:
import pandas as pd 
from glob import glob
import os
import requests

import warnings
warnings.filterwarnings("ignore")

parquet = r"https://raw.githubusercontent.com/Huynh-Tr/report/main/dthu.parquet"

def dthu():
    df = pd.read_parquet(parquet)
    df = df[df["Month year"] != "Month year"]
    df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
    lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
    lv3 = ["CAO"]
    lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
    account_code = [51131000, 51132000]
    col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", \
        "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]

    df_tsv = df.copy()
    df_tsv = df_tsv[~df_tsv["Product Group 2 Name"].isin(lv2)]
    df_tsv = df_tsv[~df_tsv["Product Group 3 Name"].isin(lv3)]
    df_tsv = df_tsv[~df_tsv["Product Group 4 Name"].isin(lv4)]
    df_tsv = df_tsv[~df_tsv["GL Account Code"].isin(account_code)]
    df_tsv = df_tsv[col]

    df_vm = df.copy()
    df_vm = df_vm[df_vm["Product Group 4 Name"].isin(lv4)]
    df_vm = df_vm[col]

    return df_tsv, df_vm

month = [5]
year = [2024]
def chitieu(year=year, month=month, plant_code='Select All'):    
    data_tsv, data_vm = dthu()
    if plant_code == 'Select All':
        sum_fm_tsv01 = data_tsv[(data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01 = data_vm[(data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07 = data_tsv[(data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07 = data_vm[(data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9

        # lk
        sum_fm_tsv01_lk = data_tsv[(data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01_lk = data_vm[(data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07_lk = data_tsv[(data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07_lk = 0 # data_vm[(data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9

        # ck
        sum_fm_tsv01_ck = data_tsv[(data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01_ck = data_vm[(data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07_ck = data_tsv[(data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07_ck = data_vm[(data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9

        # lk ck
        sum_fm_tsv01_lk_ck = data_tsv[(data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01_lk_ck = data_vm[(data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07_lk_ck = data_tsv[(data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07_lk_ck = data_vm[(data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9
        
    else:
        sum_fm_tsv01 = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01 = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07 = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07 = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9

        # lk
        sum_fm_tsv01_lk = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01_lk = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07_lk = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07_lk = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9

        # ck
        sum_fm_tsv01_ck = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01_ck = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07_ck = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(month)) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07_ck = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(month)) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9

        # lk ck
        sum_fm_tsv01_lk_ck = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01_lk_ck = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07_lk_ck = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_tsv["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07_lk_ck = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.month.isin(range(1, max(month) + 1))) & (data_vm["Month year"].dt.year.isin([max(year) - 1]))]["FM 07_Gross Profit"].sum() / 1e9

    result_dthu = pd.DataFrame(
        {
        'Chỉ Tiêu': ['Doanh thu', 'TSV', 'VM', 'Lãi Gộp', 'TSV', 'VM'],
        '': ['', '', '', '', '', ''],
        '  Thực Hiện  ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
        '  Cùng Kỳ    ': [(sum_fm_tsv01_ck + sum_fm_vm01_ck), (sum_fm_tsv01_ck), (sum_fm_vm01_ck), (sum_fm_tsv07_ck + sum_fm_vm07_ck), (sum_fm_tsv07_ck), (sum_fm_vm07_ck)],
        '  Kế Hoạch   ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
        ' ': ['', '', '', '', '', ''],
        ' LK Thực Hiện': [(sum_fm_tsv01_lk + sum_fm_vm01_lk), (sum_fm_tsv01_lk), (sum_fm_vm01_lk), (sum_fm_tsv07_lk + sum_fm_vm07_lk), (sum_fm_tsv07_lk), (sum_fm_vm07_lk)],
        ' LK Cùng Kỳ  ': [(sum_fm_tsv01_lk_ck + sum_fm_vm01_lk_ck), (sum_fm_tsv01_lk_ck), (sum_fm_vm01_lk_ck), (sum_fm_tsv07_lk_ck + sum_fm_vm07_lk_ck), (sum_fm_tsv07_lk_ck), (sum_fm_vm07_lk_ck)],
        ' LK Kế Hoạch ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
        '  ': ['', '', '', '', '', ''],
        'LK Thực Hiện ': [(sum_fm_tsv01_lk + sum_fm_vm01_lk), (sum_fm_tsv01_lk), (sum_fm_vm01_lk), (sum_fm_tsv07_lk + sum_fm_vm07_lk), (sum_fm_tsv07_lk), (sum_fm_vm07_lk)],
        '       KH Năm': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)]
        }
    )    

    return result_dthu

    # return sum_fm_tsv01, sum_fm_vm01, sum_fm_tsv07, sum_fm_vm07, sum_fm_tsv01_lk, sum_fm_vm01_lk, sum_fm_tsv07_lk, sum_fm_vm07_lk, \
    #     sum_fm_tsv01_ck, sum_fm_vm01_ck, sum_fm_tsv07_ck, sum_fm_vm07_ck, sum_fm_tsv01_lk_ck, sum_fm_vm01_lk_ck, sum_fm_tsv07_lk_ck, sum_fm_vm07_lk_ck
    

result_dthu = chitieu(year, month, plant_code='Select All')

# result_dthu = pd.DataFrame(
#     {
#         'Chỉ Tiêu': ['Doanh thu', 'TSV', 'VM', 'Lãi Gộp', 'TSV', 'VM'],
#         '': ['', '', '', '', '', ''],
#         '  Thực Hiện  ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
#         '  Cùng Kỳ    ': [(sum_fm_tsv01_ck + sum_fm_vm01_ck), (sum_fm_tsv01_ck), (sum_fm_vm01_ck), (sum_fm_tsv07_ck + sum_fm_vm07_ck), (sum_fm_tsv07_ck), (sum_fm_vm07_ck)],
#         '  Kế Hoạch   ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
#         ' ': ['', '', '', '', '', ''],
#         ' LK Thực Hiện': [(sum_fm_tsv01_lk + sum_fm_vm01_lk), (sum_fm_tsv01_lk), (sum_fm_vm01_lk), (sum_fm_tsv07_lk + sum_fm_vm07_lk), (sum_fm_tsv07_lk), (sum_fm_vm07_lk)],
#         ' LK Cùng Kỳ  ': [(sum_fm_tsv01_lk_ck + sum_fm_vm01_lk_ck), (sum_fm_tsv01_lk_ck), (sum_fm_vm01_lk_ck), (sum_fm_tsv07_lk_ck + sum_fm_vm07_lk_ck), (sum_fm_tsv07_lk_ck), (sum_fm_vm07_lk_ck)],
#         ' LK Kế Hoạch ': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)],
#         '  ': ['', '', '', '', '', ''],
#         'LK Thực Hiện ': [(sum_fm_tsv01_lk + sum_fm_vm01_lk), (sum_fm_tsv01_lk), (sum_fm_vm01_lk), (sum_fm_tsv07_lk + sum_fm_vm07_lk), (sum_fm_tsv07_lk), (sum_fm_vm07_lk)],
#         '       KH Năm': [(sum_fm_tsv01 + sum_fm_vm01), (sum_fm_tsv01), (sum_fm_vm01), (sum_fm_tsv07 + sum_fm_vm07), (sum_fm_tsv07), (sum_fm_vm07)]
#     }
# )

result_dthu

,Chỉ Tiêu,,Thực Hiện,Cùng Kỳ,Kế Hoạch,,LK Thực Hiện,LK Cùng Kỳ,LK Kế Hoạch,,LK Thực Hiện,KH Năm
0,Doanh thu,,237.873087,163.734388,237.873087,,1.461734e+03,1041.408330,237.873087,,1.461734e+03,237.873087
1,TSV,,129.918365,121.280052,129.918365,,8.400183e+02,728.988884,129.918365,,8.400183e+02,129.918365
2,VM,,107.954721,42.454336,107.954721,,6.217155e+02,312.419446,107.954721,,6.217155e+02,107.954721
3,Lãi Gộp,,35.864402,30.470322,35.864402,,3.136414e+10,186.195150,35.864402,,3.136414e+10,35.864402
4,TSV,,30.156581,29.107980,30.156581,,1.993522e+02,175.919728,30.156581,,1.993522e+02,30.156581
5,VM,,5.707821,1.362342,5.707821,,3.136414e+10,10.275422,5.707821,,3.136414e+10,5.707821


In [45]:
max([1, 2, 4]) + 1

5

# Parquet

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from glob import glob  # For handling multiple CSV files
import os

In [156]:
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"

csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory
# create a directory to save parquet files
if not os.path.exists(os.path.join(path, ".\parquet_files")):
    os.makedirs(os.path.join(path, ".\parquet_files"))

parquet_directory = os.path.join(path, "dthu_parquet_files")

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    table = pa.Table.from_pandas(df)

    # Create parquet filename based on csv filename:
    parquet_file = parquet_directory + "\\" + os.path.basename(csv_file).replace('.csv', '.parquet')

    pq.write_table(table, parquet_file)


In [ ]:


# if not os.path.exists(os.path.join(path, ".\parquet_files")):
#     os.makedirs(os.path.join(path, ".\parquet_files"))

# parquet_directory = os.path.join(path, "parquet_files\\file.parquet")

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from glob import glob
import os

def csv_to_parquet_append_legacy(csv_files, parquet_file):  # Legacy method
     if not os.path.exists(parquet_file):
         try:
             first_df = pd.read_csv(csv_files)
             table = pa.Table.from_pandas(first_df)
             pq.write_table(table, parquet_file)
         except FileNotFoundError:
             print(f"Error: First CSV file '{csv_files}' not found. Cannot create Parquet file.")
             return
         except Exception as e:
             print(f"Error creating initial Parquet file: {e}")
             return

     for csv_file in csv_files:
         try:
             df = pd.read_csv(csv_file)
             table = pa.Table.from_pandas(df)

             if os.path.exists(parquet_file):  # Read existing Parquet file
                 existing_table = pq.read_table(parquet_file)
                 combined_table = pa.concat_tables([existing_table, table]) # Concatenate tables
                 pq.write_table(combined_table, parquet_file) # Write combined table back

             else: #If file doesn't exist, write the table directly
                 pq.write_table(table, parquet_file)

         except FileNotFoundError:
             print(f"Error: CSV file '{csv_file}' not found.")
         except pa.lib.ArrowInvalid as e:
             print(f"Error converting CSV to Parquet: {e}. Check data types for file: {csv_file}")
         except Exception as e:
             print(f"An unexpected error occurred while processing {csv_file}: {e}")

# Example usage:
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory

parquet_file = 'output.parquet'  # Replace with your desired Parquet file path

csv_to_parquet_append(csv_files, parquet_file)

print(f"CSV files appended to Parquet file: {parquet_file}")

<>:48: SyntaxWarning: invalid escape sequence '\p'
<>:48: SyntaxWarning: invalid escape sequence '\p'
C:\Users\huynh.th\AppData\Local\Temp\ipykernel_32680\2253336765.py:48: SyntaxWarning: invalid escape sequence '\p'
  path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"


An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202101.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202102.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202103.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202104.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202105.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202106.csv: __cinit__() got an unexpected 

In [13]:
# create an empty parquet file
import pyarrow as pa
import pyarrow.parquet as pq

# Create an Arrow table with an empty schema
parquet_file = 'output.parquet'
table = pa.Table.from_pandas(pd.DataFrame())
pq.write_table(table, parquet_file)

In [47]:
import pyarrow as pa
import pyarrow.parquet as pq

# Create an Arrow table with an empty schema
parquet_file = 'output.parquet'
table = pa.Table.from_pandas(pd.DataFrame())
# pq.write_table(table, parquet_file)

# Path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory

pd_df = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files])
table = pa.Table.from_pandas(pd_df)
pq.write_table(table, parquet_file)